In [1]:
import duckdb
import pandas as pd

In [2]:
# create prod db

sql_query_import_1 = '''
CREATE OR REPLACE TABLE parking_violation_codes AS
SELECT *
FROM read_csv_auto(
  'data/dof_parking_violation_codes.csv',
  normalize_names=True
  )
'''

sql_query_import_2 = '''
CREATE OR REPLACE TABLE parking_violations_2023 AS
SELECT *
FROM read_csv_auto(
  'data/parking_violations_issued_fiscal_year_2023_sample.csv',
  normalize_names=True
  )
'''

with duckdb.connect('data/prod_nyc_parking_violations.db') as con:
  con.sql(sql_query_import_1)
  con.sql(sql_query_import_2)



In [4]:
sql_query = '''
  select * from "nyc_parking_violations"."main_dbt_test__audit"."violation_codes_revenue"
'''

show_tables = '''
show tables
'''

show_bronze_codes = '''
select *
from bronze_parking_violation_codes
limit 10
'''

show_bronze_violations_count = '''
select violation_county as County, 
count (distinct violation_county) as Count
from bronze_parking_violations
where violation_county is not null
group by County
'''
show_bronze_violations = ''' 
SELECT violation_county, COUNT(violation_county) 
FROM bronze_parking_violations
GROUP BY violation_county
ORDER BY COUNT(violation_county) DESC
'''


# SELECT program_type  AS [Type],
#        Count(DISTINCT program_name) AS [Count],
# FROM   cm_production
# WHERE  push_number = @push_number
# GROUP  BY program_type 


# prod
with duckdb.connect('data/prod_nyc_parking_violations.db') as con:
    display(con.sql(show_tables).df())

,name
0,bronze_parking_violation_codes
1,bronze_parking_violations
2,gold_ticket_metrics
3,gold_vehicles_metrics
4,parking_violation_codes
5,parking_violations_2023
6,silver_violation_tickets
7,silver_violation_vehicles


In [15]:
silver_query = '''
SELECT
        violation_code,
        definition,
        TRUE AS is_manhattan_96th_st_below,
        manhattan_96th_st_below AS fee_usd,
    FROM
        bronze_parking_violation_codes
'''

silver = '''
SELECT *
FROM silver_violation_vehicles
LIMIT 250
'''

with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql(silver).df())

,summons_number,registration_state,plate_type,vehicle_body_type,vehicle_make,vehicle_expiration_date,vehicle_color,vehicle_year
0,9010912681,CA,PAS,SUBN,FORD,20220788,BLACK,0
1,4858762841,NY,PAS,4DSD,HONDA,0,GY,2003
2,4854645684,FL,PAS,UT,BMW,0,WHI,2022
3,9044582707,NY,PAS,SUBN,SUBAR,20231217,GY,2017
4,9041503330,NY,PAS,4DSD,CHEVR,20250320,BK,2018
...,...,...,...,...,...,...,...,...
245,9019388216,NY,PAS,4DSD,ACURA,20230203,WH,2021
246,9034573242,NY,COM,VAN,FORD,20240930,WH,2020
247,4843867718,NY,PAS,SUBN,ROVER,0,BK,2023
248,8977284028,NY,PAS,SUBN,CHEVR,20240530,BL,1997


In [3]:
gold_query = ''' 
SELECT *
FROM gold_vehicles_metrics'''

'''
SELECT
        violation_code,
        definition,
        TRUE AS is_manhattan_96th_st_below,
        manhattan_96th_st_below AS fee_usd,
    FROM
        bronze_parking_violation_codes
'''


with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql(gold_query).df())

,registration_state,ticket_count
0,NJ,9258
1,PA,3514
2,FL,2414
3,CT,1787
4,GA,840
...,...,...
57,AB,1
58,BC,1
59,NB,1
60,PE,1
